<center><h1>Linear Regression Models in R</h1></center>


# 1 Linear Regression Models

<center><img src="images/age_height.png" width=720 /></center>

<center><img src="images/age_height_lm.png" width=720 /></center>

In [ ]:
## Below is the code for simulating the child height, weight, and age data

library(ggplot2)
library(plotly)

set.seed(137)

n <- 500

beta0 <- 80
beta1 <- 3.5
beta2 <- 2.0

age <- rnorm(n, 3.5, 1.5)
weight <- rnorm(n, 16, 3)
height <- beta0 + beta1*age + beta2*weight + rnorm(n, 8, 3.5)

ds <- data.frame(age, weight, height)

## We can use the plot_ly() function to generate a 3-d plot
plot_ly(ds, x = ~age, y = ~weight, z = ~height, marker = list(symbol = 'circle', 
                                                              size = 5, 
                                                              color = "blueviolet"))


## 1.1 Linear Regression Models (Review)
        
$$ y_i = \beta_0 + \beta_1 x_{i1} + \beta_2 x_{i2} + ... + \beta_p x_{ip} + \varepsilon_i $$  
  - Outcome variable ($y$) is continuous
  - Can have one or many predictor variables
  - Predictors can be continuous or categorical
  - Examples: 
    + Estimating effect square footage on home price             
    + Effect of age and weight on blood pressure


### 1.1.1 Linear Regression Models (cont.)
 $$ \text{height}_i = \beta_0 + \beta_1 \text{age}_i + \varepsilon_i $$  
 
$\texttt{height}_i$: dependent/outcome variable

$\texttt{age}_i$: predictor variable

## 1.2 Assumptions of Linear Regression Models
                

- $E(y_i) = \mu_i = \beta_0 + \beta_1 x_i $ 
  + Or equivalently $E(\varepsilon_i) = 0$
  + The means of $E(y_i)$ are on a straight line
- $var(y_i) = \sigma^2$
  + Or equivalently $var(\varepsilon_i) = \sigma^2$
  + Known as _homoscedasticity_
- $cov(y_i, y_j) = 0$ 
  + Or equivalently $cov(\varepsilon_i, \varepsilon_j) = 0$
  + Errors are uncorrelated
- $\varepsilon_i$ is normally distributed
  + Needed when using maximum likelihood estimation (MLE), but not ordinary least squares (OLS)


## 1.3 Single-Variable Linear Regression
Suppose we are interested in predicting the total area destroyed by wildfires using air temperature measurements. We can do this using linear regression.

In [ ]:
library(ggplot2)  

fires_df <- read.csv("data/montesinho_forestfires.csv")

head(fires_df)

### 1.3.1 Plotting Data (_always!!_)

In [ ]:
ggplot(fires_df, aes(x = temp, y = area)) +
    geom_point(colour = "blueviolet", alpha = 0.4) +
    ylim(-5, 50)

### 1.3.2 The `lm()` Function in R

  - Performs linear regression
  - Create a fitted linear model object, which contains resultss

In [ ]:
fm1 <- lm(area ~ temp, fires_df)   # perform linear regression

summary(fm1)                       # see output of regression

### 1.3.3 Using `tidy()` Instead of `summary()`

- `tidy()` function from _broom_ package gives us (arguably) nicer output
- Newer, and somewhat less common, but part of _tidyverse_ ecosystem (e.g., _dplyr_, _ggplot2_)

In [ ]:
library(broom)

tidy(fm1)

## 1.4 Multivariate Linear Regression 

  - Regression modeles can have arbitrary number of predictor variales
  - Simply add variable in the formula (e.g., `y ~ x1 + x2 + x3`)

In [ ]:
fm2 <- lm(area ~ temp + rain, fires_df)

tidy(fm2)

## 2. Model Fit and Diagnostics

 - How "good" is our model?
 - Which model is "better"?

## 2.1 Interpretting $R^2$

- Represents the proportion of variance explained
- Sometimes called "_coefficient of determination_"

$$R^{2} = 1 - \frac{SSE}{SST}$$

where we have,

$SSE = \sum_{i}^{n} \left( y_{i} - \hat{y_{i}} \right) ^{2}$, and 

$SST = \sum_{i}^{n} \left( y_{i} - \bar{y} \right) ^{2}$

### 2.1.1 $R^2$ with `glance()` 

In [ ]:
glance(fm1)        # using glance() on our fitted model object

### 2.1.2 $R^2$ with `summary()`

In [ ]:
summary(fm2)

## 2.2 What is a "good" $R^2$ Value?



<center>It depends...</center>


<center>¯\_(ツ)_/¯</center>

## 2.3 Residuals and Outliers

In [ ]:
fm1_aug <- augment(fm1)

head(fm1_aug) 

### 2.3.1 Distribution of Residuals

In [ ]:
ggplot(fm1_aug, aes(x = .resid)) +
    geom_histogram(colour = "lightblue", fill = "skyblue", bins = 50) +
    xlim(-40, 40)

### 2.3.2 Cook's Distance (Cook's D)

In [ ]:
fm1_aug$row_num <- 1:nrow(fm1_aug)

ggplot(fm1_aug, aes(x = row_num, y = .cooksd)) +
    geom_point(colour = "blue") 

### 2.3.3 Residuals vs. Fitted Values

In [ ]:
ggplot(fm1_aug, aes(x = .fitted, y = .resid)) +
    geom_point(colour = "blue") 

In [ ]:
library(dplyr)
library(stringr)

arrests_df <- read.csv("data/pvd_arrests_2021-10-03.csv", na.strings = c("Unknown", "NULL", ""))

<center><h1>Challenge Problem</h1></center>

Suppose we are interested in what factor might contribute to the number of Police officers taking part in arresting someone. For example, maybe violent offenders tend to have more police officers involved in their arrests. 

Let's use the Providence Police Department data to test whether `age`, `gender`, and the proportion of violent offenses (i.e., `prop_violent` below) are statistically significant predictors of the average number of arresting officers (i.e., `mean_officer_cnt` below). 


The code below from our prior lesson should help us get started by creating the `person_df` dataframe. Let's use this dataframe for our model fitting.

Let's build 4 models, one with each predictor (i.e., `age`, `gender`, and `prop_violent`) regressed on our outcome variable (i.e., `mean_officer_cnt`) and then one with all three predictors regressed on `mean_officer_cnt`. What do we observe? Are there statistically significant effects?


In [ ]:
is_uppercase <- function(chr) {
    res <- chr %in% LETTERS
    return(res)
}

has_full_names <- function(names_str) {
    char1 <- str_sub(names_str, 1, 1)
    char2 <- str_sub(names_str, 2, 2)
    
    res <- !(is_uppercase(char1) && is_uppercase(char2))
    return(res)
}

In [ ]:
count_names <- function(names_str) {
    names_str_trm <- str_trim(names_str)     # remove whitespace
    
    if (has_full_names(names_str_trm)) {
        split_char <- "/ "
    } else {
        split_char <- ", "
    }
    
    name_list <- str_split(names_str_trm, split_char)
    
    name_vec <- unlist(name_list)
    
    k <- length(name_vec)
    
    return(k)
}


count_all_names <- function(col) {

    n <- length(col)   # get the length of our input column
    cnts <- rep(0, n)  # allocate vector of zeros to populate with counts

    for (i in 1:n) {
        cnts[i] <- count_names(col[i])
    }
    return(cnts) 
}

In [ ]:
is_violent_offense <- function(v) {

    violent_terms <- c("domestic-asslt", "assault", "battery", "murder")
    n_obs <- length(v)
    is_violent <- rep(FALSE, n_obs)
    
    # iterate over all statute descriptions
    for (i in 1:n_obs) {
        
        # iterate over the 4 terms associated with violence
        for (term in violent_terms) {
            if (!is.na(v[i]) && str_detect(tolower(v[i]), term)) {

                is_violent[i] <- TRUE
            }
        }
    }
    return(is_violent)
}

In [ ]:
arrests_df$violent <- is_violent_offense(arrests_df$statute_desc)
arrests_df$officer_cnt <- count_all_names(arrests_df$arresting_officers)

In [ ]:
person_df <- arrests_df %>%
    group_by(id) %>%
    summarise(
        total_charges = n(),
        num_uniq_arrests = length(unique(case_number)),
        prop_violent = mean(violent),
        mean_officer_cnt = mean(officer_cnt),
        age = age[1],
        gender = gender[1]
    ) 